In [10]:
# Disable info messages from the tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom
import pickle

In [11]:

#Load train data
with open('train', 'rb') as fo:         #Open file
    d = pickle.load(fo, encoding='bytes')       #Load pickle data
    train_data = d['data']                      #Load train data
    train_label = d['label']                    #Load train label
 
 
with open('test', 'rb') as fo:          #Open file
    d = pickle.load(fo, encoding='bytes')       #Load pickle data
    test_data = d['data']                       #Load test data
    test_label = d['label']                     #Load test data
#print(train_data.shape,test_data.shape) # (84420,310) # (58128,310)
#Merge train and test data in order to scale them
X = np.concatenate([train_data,test_data])
# scaling the inputs from 0 to 1
X = scale(X)
Y = np.concatenate([train_label,test_label])
classes = len(np.unique(Y))
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape) #(142548,310)

(142548, 310) (142548, 3)


In [12]:
# split into train and test data
X_train = X[:84420]
X_test =  X[84420:]
Y_train = Y[:84420]
Y_test = Y[84420:]

In [13]:
y_train = np.argmax(Y_train,axis=-1)
from tab2img.converter import Tab2Img
model = Tab2Img()
X_train = model.fit_transform(X_train, y_train)
X_test = model.transform(X_test)

In [14]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(X_train)):
    img = Image.fromarray(X_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')

for i in range(len(X_test)):
    img = Image.fromarray(X_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')

print(x_train.shape,x_test.shape)

(84420, 32, 32, 3) (58128, 32, 32, 3)


In [15]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu',name='flattenn')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
330/330 [==============================] - 33s 99ms/step - loss: 0.2310 - accuracy: 0.9065 - val_loss: 2.4120 - val_accuracy: 0.6154
Epoch 2/20
330/330 [==============================] - 26s 78ms/step - loss: 0.0232 - accuracy: 0.9931 - val_loss: 1.4349 - val_accuracy: 0.7247
Epoch 3/20
330/330 [==============================] - 26s 78ms/step - loss: 0.0089 - accuracy: 0.9976 - val_loss: 2.0177 - val_accuracy: 0.7079
Epoch 4/20
330/330 [==============================] - 26s 78ms/step - loss: 0.0165 - accuracy: 0.9955 - val_loss: 1.7762 - val_accuracy: 0.7308
Epoch 5/20
330/330 [==============================] - 26s 78ms/step - loss: 0.0114 - accuracy: 0.9970 - val_loss: 1.6538 - val_accuracy: 0.7249
Epoch 6/20
330/330 [==============================] - 26s 78ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 1.8737 - val_accuracy: 0.7182
Epoch 7/20
330/330 [==============================] - 26s 79ms/step - loss: 4.9880e-04 - accuracy: 0.9999 - val_loss: 1.9591 - val_accur

In [16]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

2639/2639 [==============================] - 33s 13ms/step - loss: 3.7717e-09 - accuracy: 1.0000
training accuracy:  1.0
testing accuracy:  0.7525976896286011
